# Convert Caffe Model to Tensorflow

This is a somewhat cumbersome process, and I've probably introduced several inefficiencies along the way. However, the final product -- a tensorflow server -- works like a charm. This notebook is primarily concerned with converting a caffe model to a tensorflow 1 protobuf. The notebook that follows this converts the tf1 model into a tf2 servable docker image and tests it. This notebook uses tf1 and the notebook that follows this uses tf2.

## Dev Environment

Before running this notebook, create a new conda environment with python2.7: `conda create --name convert_caffe python=2.7 -y`

In [ ]:
# Important: make sure the right version of tensorflow is installed.
! conda install tensorflow==1.15.0 protobuf jupyterlab

In [1]:
# Test the installations.
import os
import tensorflow as tf
from tensorflow.python.platform import gfile

# Tensorflow 1 should be used -- I have not tested tf2!
assert tf.__version__ == '1.15.0'

'1.15.0'

## Model Acquisition

I used VGG-16 trained on IMDB Wiki. See the gender section of [this site](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/).

In [2]:
! curl https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/gender.caffemodel -O

In [ ]:
! curl https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/gender.prototxt -O

## Caffe to pb Conversion

I followed [this tutorial](https://ndres.me/post/convert-caffe-to-tensorflow/). Here are the relevant commands:

In [ ]:
# Clone the repo
! git clone https://github.com/linkfluence/caffe-tensorflow
! cd caffe-tensorflow

In [ ]:
# Troubleshooting: you might have to do this:
! pip uninstall tensorflow protobuf && pip install tensorflow==1.15.0 protobuf
! pip install numpy==1.16.1

In [ ]:
# Takes approx 5 minutes on my crappy Macbook
! python convert.py --caffemodel ../gender.caffemodel ../gender.prototxt
! --data-output-path ./output.mat --code-output-path ./output2.py --standalone-output-path ./gender_model.pb

## Pb to Servable Conversion

In [34]:
# Save the file as a servable.

from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

export_dir = "./gender_model"
graph_pb = "gender_model.pb"

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

sigs = {}

with tf.Session(graph=tf.Graph()) as sess:
    # name="" is important to ensure we don't get spurious prefixing
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
    inp = g.get_tensor_by_name("data:0")
    out = g.get_tensor_by_name("prob:0")

    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.saved_model.signature_def_utils.predict_signature_def(
            {"in": inp}, {"out": out})
    
    tf.saved_model.simple_save(sess,
            export_dir,
            inputs={"input": inp},
            outputs={"output": out})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./gender_model/saved_model.pb


## Next Steps

See [the next notebook]() for the remaining steps.